In [1]:
# imports
import pandas as pd
import matplotlib.pyplot

In [2]:
# read files 
private_data_original = pd.read_csv('../data/private_dataB.csv')
public_data_register = pd.read_csv('../data/public_data_registerB.csv')
public_data_results = pd.read_csv('../data/public_data_resultsB.csv')

## Doin transforms you know they did too

In [3]:
public_data_register.head()

,Unnamed: 0,name,sex,dob,zip,citizenship,marital_status,last_voted
0,0,"Le, Ian",Male,1998-08-30,2300,Denmark,Never married,1
1,1,"Wang, Caleb",Male,1996-04-23,2200,Denmark,Never married,1
2,2,"Carranco, Raquel",Female,1994-08-06,2200,Denmark,Never married,1
3,3,"Williams, Geona",Female,1995-04-22,2100,Sweden,Never married,2
4,4,"Gonzales, Edward",Male,2000-05-13,2100,Denmark,Never married,1
